In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import datetime as dt

import utils

import calitp
from calitp.tables import tbl
from siuba import *

# Replace get_recent_dates()
# Explicitly set dates

dates = {
    'thurs': dt.date(2021, 10, 7),
    'sat': dt.date(2021, 10, 9),
    'sun': dt.date(2021, 10, 10)
}

min_date = min(dates.values())
max_date = max(dates.values())

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# store temporarily here
DATA_PATH = "gs://calitp-analytics-data/data-analyses/bus_service_increase/test/"
#thurs = pd.read_parquet(f"{DATA_PATH}trips_thurs.parquet")
#sat = pd.read_parquet(f"{DATA_PATH}trips_sat.parquet")
#sun = pd.read_parquet(f"{DATA_PATH}trips_sun.parquet")

#df = pd.concat([thurs, sat, sun], axis=0, ignore_index=True)
#df.to_parquet(f"{DATA_PATH}all_days_st2.parquet")

df = pd.read_parquet(f"{DATA_PATH}all_days_st2.parquet")

In [3]:
def get_time_calculations(df):
    ## time calculations
    df = df.assign(
        date = pd.to_datetime(df.date),
        departure_time = df.departure_time.dropna().apply(utils.fix_gtfs_time),
    )

    # Something weird comes up trying to generate departure_dt
    # pd.to_datetime() gives today's date
    # datetime.strptime gives year 1900
    # Either way, we have the service date, and later subsetting between 5am-9pm will address this
    df = df.assign(
        departure_time = pd.to_datetime(df.departure_time),
        departure_hour = pd.to_datetime(df.departure_time).dt.hour,
    )
    
    # Any observation with NaTs for departure time get dropped
    # Will create issue later when grouping is done with departure hour
    df = df[df.departure_time.notna()].reset_index(drop=True)
    
    return df


def calculate_runtime_hourlytrips(df):
    # Calculate run time for a trip
    # Find the median stop (keep that observation)
    group_cols = ['trip_key', 'day_name']
    df = df.assign(
        mindt = df.groupby(group_cols)["departure_time"].transform("min"),
        maxdt = df.groupby(group_cols)["departure_time"].transform("max"),
        middle_stop = df.groupby(["trip_key", "day_name"])["stop_sequence"].transform("median"),
    ).astype({"middle_stop": "int64"})

    df = df.assign(
        runtime_seconds = (df.maxdt - df.mindt).dt.seconds
    ).drop(columns = ["mindt", "maxdt"])
    
    # Drop any trips with runtime of NaN calculated
    df = df[df.runtime_seconds.notna()].reset_index(drop=True)

    # Still want to use this to merge on the mean runtime info
    middle_stops = df >> filter(_.stop_sequence == _.middle_stop)
    
    middle_stops = middle_stops.assign(
        mean_runtime_min = (middle_stops.groupby(["calitp_itp_id", 
                                                  "route_id", "shape_id", 
                                                  "departure_hour", "day_name"])
                            ["runtime_seconds"].transform("mean")
                           )
    )
    
    debug_me = middle_stops[middle_stops.mean_runtime_min.isna()][
        ["calitp_itp_id", "shape_id", "trip_key"]]
    print("Debug errors for NaN mean runtimes")
    print(debug_me.head())
    # Why are there some NaNs from this, when NaNs were dropped before?
    # Some are due to no departure_time (handle it above by dropping NaTs)
    
    middle_stops = middle_stops.assign(
        mean_runtime_min = (middle_stops.mean_runtime_min.dropna()     
                            .apply(lambda x: int(round(x) / 60))
                           )
    )   
    
    # Add trips per hour column
    shape_frequency = (
        middle_stops
        >> count(_.calitp_itp_id, _.route_id,
                 _.shape_id, _.departure_hour, _.day_name, sort = True)
        >> rename(trips_per_hour = "n")
        >> inner_join(_, middle_stops, 
                      on = ["calitp_itp_id", "day_name", 
                            "shape_id", "departure_hour", "route_id"])
    )
    
    # Now, data is at the trip-level (trip_key) still present
    # Drop duplicates, but no aggregation because trips_per_hour and mean_runtime 
    # are already correctly generated at the route-level, across trips in that departure hour
    shape_frequency = shape_frequency.drop_duplicates(subset=[
        "calitp_itp_id", "shape_id", "departure_hour",
        "day_name", "route_id"])
    
    # There's an aggregation to deal with multiple route_ids that share same shape_id
    # If there are still multiple route_ids, then aggregate and sum / mean
    # Modify this to include itp_id into the groupby
    shape_frequency2 = (shape_frequency.groupby(
        ["calitp_itp_id", "shape_id", "departure_hour", "day_name"])
                        .agg({"route_id": "max", 
                              "trips_per_hour": "sum", 
                              "mean_runtime_min": "mean"
                             }).reset_index()
                       )
    
    # Now, drop ITP_ID==200 to use individual operator feeds
    shape_frequency3 = shape_frequency2 >> filter(_.calitp_itp_id != 200)
    
    return shape_frequency3


def attach_funding(all_operators_df):
    # This is a small query, can leave it here
    with_funding = (tbl.views.transitstacks()
                    >> select(_.calitp_itp_id == _.itp_id, _.ntd_id, 
                              _.transit_provider, _._5307_funds, _._5311_funds,
                              _.operating_expenses_total_2019)
                    >> collect()
                    >> right_join(_, all_operators_df, on = 'calitp_itp_id')
                   )
    
    def fix_funds(value):
        if type(value) != str:
            return None
        else:
            return int(value.replace('$', '').replace(',', ''))
        
    funding_cols = ["_5307_funds", "_5311_funds", "operating_expenses_total_2019"] 
    for c in funding_cols:
        with_funding[c] = with_funding[c].apply(fix_funds)
    
    return with_funding

In [4]:
## Debug and check some of the cases 
# Just look at LA Metro for now
# See if the 30 trips per hour comes up again
df = pd.read_parquet(f"{DATA_PATH}all_days_st2.parquet")
df = df[df.calitp_itp_id==182]

df = get_time_calculations(df)

In [5]:
df

,calitp_itp_id,date,trip_key,trip_id,is_in_service,day_name,stop_sequence,departure_time,stop_id,shape_id,route_id,departure_hour
0,182,2021-10-07,861585063715735270,53810853-SEPT21-D98-1_Weekday,True,Thursday,65,2022-02-02 09:01:00,4557,1670098_SEPT21,167-13149,9
1,182,2021-10-07,7093655288726676976,53964976-SEPT21-D05CAR-1_Weekday,True,Thursday,61,2022-02-02 10:19:00,12864,2120213_SEPT21,212-13149,10
2,182,2021-10-07,7701903612960011416,53933344-SEPT21-D18CAR-1_Weekday,True,Thursday,60,2022-02-02 18:42:00,6637,1170075_SEPT21,117-13149,18
3,182,2021-10-07,2914452950964544558,53934624-SEPT21-D18CAR-1_Weekday,True,Thursday,62,2022-02-02 08:30:00,4838,2100302_SEPT21,210-13149,8
4,182,2021-10-07,1149136744223468836,54015407-SEPT21-D02CAR-1_Weekday,True,Thursday,71,2022-02-02 10:03:00,5932,6110029_SEPT21,611-13149,10
...,...,...,...,...,...,...,...,...,...,...,...,...
2195209,182,2021-10-10,-3699170711897233212,53945118-SEPT21-D09CAR-3_Sunday,True,Sunday,57,2022-02-02 16:35:00,9218,780436_SEPT21,78-13149,16
2195210,182,2021-10-10,7172571937594546479,53836803-SEPT21-D08CAR-3_Sunday,True,Sunday,57,2022-02-02 11:27:00,15212,1650198_SEPT21,165-13149,11
2195211,182,2021-10-10,-174811612801981664,53934020-SEPT21-D18CAR-3_Sunday,True,Sunday,57,2022-02-02 16:24:00,9375,1270039_SEPT21,127-13149,16
2195212,182,2021-10-10,-3751852536085031234,53945068-SEPT21-D09CAR-3_Sunday,True,Sunday,57,2022-02-02 10:12:00,12288,780421_SEPT21,78-13149,10


In [6]:
def to_debug(df):
    group_cols = ['trip_key', 'day_name']
    df = df.assign(
        mindt = df.groupby(group_cols)["departure_time"].transform("min"),
        maxdt = df.groupby(group_cols)["departure_time"].transform("max"),
        middle_stop = df.groupby(["trip_key", "day_name"])["stop_sequence"].transform("median"),
    ).astype({"middle_stop": "int64"})

    df = df.assign(
        runtime_seconds = (df.maxdt - df.mindt).dt.seconds
    ).drop(columns = ["mindt", "maxdt"])

    # Still want to use this to merge on the mean runtime info
    middle_stops = df >> filter(_.stop_sequence == _.middle_stop)
    
    middle_stops = middle_stops.assign(
        mean_runtime_min = (middle_stops.groupby(["calitp_itp_id", 
                                                  "route_id", "shape_id", 
                                                  "departure_hour", "day_name"])
                            ["runtime_seconds"].transform("mean")
                            .apply(lambda x: int(round(x) / 60)))
    )
    
    # Add trips per hour column
    shape_frequency = (
        middle_stops
        >> count(_.calitp_itp_id, _.route_id,
                 _.shape_id, _.departure_hour, _.day_name, sort = True)
        >> rename(trips_per_hour = "n")
        >> inner_join(_, middle_stops, 
                      on = ["calitp_itp_id", "day_name", 
                            "shape_id", "departure_hour", "route_id"])
    )
    '''
    # Now, data is at the trip-level (trip_key) still present
    # Drop duplicates, but no aggregation because trips_per_hour and mean_runtime 
    # are already correctly generated at the route-level, across trips in that departure hour
    shape_frequency = shape_frequency.drop_duplicates(subset=[
        "calitp_itp_id", "shape_id", "departure_hour",
        "day_name", "route_id"])
    
    # There's an aggregation to deal with multiple route_ids that share same shape_id
    # If there are still multiple route_ids, then aggregate and sum / mean
    # Modify this to include itp_id into the groupby
    shape_frequency2 = (shape_frequency.groupby(
        ["calitp_itp_id", "shape_id", "departure_hour", "day_name"])
                        .agg({"route_id": "max", 
                              "trips_per_hour": "sum", 
                              "mean_runtime_min": "mean"
                             }).reset_index()
                       )
    '''
    return shape_frequency

In [7]:
df2 = to_debug(df)

group_cols = ["calitp_itp_id", "shape_id", "departure_hour", "day_name", "route_id"]
df2 = df2.assign(
    obs = df2.groupby(group_cols)["trip_key"].cumcount() + 1,
)

df2 = df2.assign(
    max_obs = df2.groupby(group_cols)["obs"].transform("max")
)

In [8]:
df2.max_obs.max()

19

In [9]:
df2[df2.max_obs==19]

,calitp_itp_id,route_id,shape_id,departure_hour,day_name,trips_per_hour,date,trip_key,trip_id,is_in_service,stop_sequence,departure_time,stop_id,middle_stop,runtime_seconds,mean_runtime_min,obs,max_obs
0,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,-3688991411338276459,54133711-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:26:00,30022,1,540,9,1,19
1,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,8297285008995166099,54133713-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:32:00,30022,1,540,9,2,19
2,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,-6494341673185312199,54133717-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:44:00,30022,1,540,9,3,19
3,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,6820499949529949431,54133722-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:59:00,30022,1,540,9,4,19
4,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,51706130366599061,54133719-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:50:00,30022,1,540,9,5,19
5,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,4406007602970273768,54133712-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:29:00,30022,1,540,9,6,19
6,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,-7005119982581110102,54133718-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:47:00,30022,1,540,9,7,19
7,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,-2414876668720152120,54133710-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:23:00,30022,1,540,9,8,19
8,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,5742717980696682034,54133705-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:08:00,30022,1,540,9,9,19
9,182,SOFI,6990001_SoFi,16,Sunday,19,2021-10-10,-2997845282772078049,54133707-SoFi_Stadium_Express_1305,True,1,2022-02-02 16:14:00,30022,1,540,9,10,19


In [10]:
# Ok, to deal with duplicates, because there are multiple trips
# with same departure hour, but that trips_per_hour is already generated correctly
# Just drop duplicates, don't need to sum it up trips_per_hour
# The mean runtime is already derived across all these trips
df3 = df2.drop_duplicates(subset=["calitp_itp_id", "shape_id", "departure_hour",
                                  "day_name", "route_id"])

In [11]:
## Aggregation to deal with multiple route_ids that share same shape_id
# It makes 3 trips per hour, same 
df3[df3.shape_id=="964395_shp"]


,calitp_itp_id,route_id,shape_id,departure_hour,day_name,trips_per_hour,date,trip_key,trip_id,is_in_service,stop_sequence,departure_time,stop_id,middle_stop,runtime_seconds,mean_runtime_min,obs,max_obs


In [12]:
keep_trips = [-7711476650844868921,
              -6084928573786923571]
silver = df2[(df2.route_id.str.contains("910")) & 
            (df2.departure_hour == 9) & (df2.trip_key.isin(keep_trips))]

In [13]:
silver.runtime_seconds.value_counts()

3540    1
Name: runtime_seconds, dtype: int64

In [14]:
silver

,calitp_itp_id,route_id,shape_id,departure_hour,day_name,trips_per_hour,date,trip_key,trip_id,is_in_service,stop_sequence,departure_time,stop_id,middle_stop,runtime_seconds,mean_runtime_min,obs,max_obs
6480,182,910-13149,9100211_SEPT21,9,Thursday,5,2021-10-07,-7711476650844868921,53936837-SEPT21-D18CAR-1_Weekday,True,11,2022-02-02 09:57:00,5040,11,3540,59,2,5


In [15]:
(3450 +5100)/2

4275.0

In [19]:
# Still want to use this to merge on the mean runtime info
middle_stops = df2 >> filter(_.stop_sequence == _.middle_stop)

In [20]:
middle_stops[middle_stops.trip_key.isin(keep_trips)]

,calitp_itp_id,route_id,shape_id,departure_hour,day_name,trips_per_hour,date,trip_key,trip_id,is_in_service,stop_sequence,departure_time,stop_id,middle_stop,runtime_seconds,mean_runtime_min,obs,max_obs
6480,182,910-13149,9100211_SEPT21,9,Thursday,5,2021-10-07,-7711476650844868921,53936837-SEPT21-D18CAR-1_Weekday,True,11,2022-02-02 09:57:00,5040,11,3540,59,2,5
22956,182,910-13149,9100209_SEPT21,10,Thursday,2,2021-10-07,-6084928573786923571,53936848-SEPT21-D18CAR-1_Weekday,True,17,2022-02-02 10:15:00,10994,17,5100,85,2,2


In [21]:
middle_stops = middle_stops.assign(
    mean_runtime_min = (middle_stops.groupby(["calitp_itp_id", "route_id", "shape_id", 
                          "departure_hour", "day_name"])
                        ["runtime_seconds"].transform("mean")
                        .apply(lambda x: int(round(x) / 60)))
)

In [22]:
# With these 2 trips, it's because the starting stop has departure hour = 9, but by 
# middle stop, it's departure hour = 10. 
# Allow departure hour to differ for trip_keys then, since we want to stick with middle stop
middle_stops[middle_stops.trip_key.isin(keep_trips)]

,calitp_itp_id,route_id,shape_id,departure_hour,day_name,trips_per_hour,date,trip_key,trip_id,is_in_service,stop_sequence,departure_time,stop_id,middle_stop,runtime_seconds,mean_runtime_min,obs,max_obs
6480,182,910-13149,9100211_SEPT21,9,Thursday,5,2021-10-07,-7711476650844868921,53936837-SEPT21-D18CAR-1_Weekday,True,11,2022-02-02 09:57:00,5040,11,3540,59,2,5
22956,182,910-13149,9100209_SEPT21,10,Thursday,2,2021-10-07,-6084928573786923571,53936848-SEPT21-D18CAR-1_Weekday,True,17,2022-02-02 10:15:00,10994,17,5100,85,2,2


In [23]:
shape_frequency = (
    middle_stops
    >> count(_.calitp_itp_id, _.route_id,
             _.shape_id, _.departure_hour, _.day_name, sort = True)
    >> rename(trips_per_hour = "n")
    >> inner_join(_, middle_stops, 
                  on = ["calitp_itp_id", "day_name", 
                        "shape_id", "departure_hour", "route_id"])
)
    

In [24]:
shape_frequency[shape_frequency.trip_key.isin(keep_trips)]

,calitp_itp_id,route_id,shape_id,departure_hour,day_name,trips_per_hour_x,trips_per_hour_y,date,trip_key,trip_id,is_in_service,stop_sequence,departure_time,stop_id,middle_stop,runtime_seconds,mean_runtime_min,obs,max_obs
6480,182,910-13149,9100211_SEPT21,9,Thursday,5,5,2021-10-07,-7711476650844868921,53936837-SEPT21-D18CAR-1_Weekday,True,11,2022-02-02 09:57:00,5040,11,3540,59,2,5
22956,182,910-13149,9100209_SEPT21,10,Thursday,2,2,2021-10-07,-6084928573786923571,53936848-SEPT21-D18CAR-1_Weekday,True,17,2022-02-02 10:15:00,10994,17,5100,85,2,2


In [25]:
multiple_shapes = (shape_frequency.groupby(["shape_id", "day_name", "departure_hour"])
                   .agg({"route_id": "nunique"})
                   .reset_index()
                  )
                  

In [26]:
all_operators_shape_frequency = pd.read_parquet(f"{utils.GCS_FILE_PATH}shape_frequency.parquet")

In [27]:
single = all_operators_shape_frequency[all_operators_shape_frequency.calitp_itp_id==279]

In [28]:
(single >> filter(_.shape_id == '964395_shp')).dropna()

,shape_id,day_name,departure_hour,calitp_itp_id,route_id,trips_per_hour,mean_runtime_min
797,964395_shp,Thursday,5,279,4,4,24.5
822,964395_shp,Saturday,6,279,4,1,24.0
848,964395_shp,Sunday,8,279,4,1,24.0


In [29]:
debug = pd.read_parquet("./data/test/timecalc_thurs.parquet")

error_trips = [
    #thurs
    4561616254186924304,
    -3573784532994111184,
    -5890052631007121734,
    # sat
    #8542179829811914215,
    #4330243390808932084,
    #-5417826296843621461
]

debug = debug[debug.trip_key.isin(error_trips)]
debug

FileNotFoundError: calitp-analytics-data/data-analyses/bus_service_increase/timecalc_thurs.parquet

In [30]:
all_operators_shape_frequency = pd.read_parquet(f"{utils.GCS_FILE_PATH}shape_frequency.parquet")
all_operators_shape_frequency[
    (all_operators_shape_frequency.calitp_itp_id.isin(debug.calitp_itp_id)) & 
    (all_operators_shape_frequency.route_id.isin(debug.route_id)) & 
    (all_operators_shape_frequency.mean_runtime_min.isna())
]

NameError: name 'debug' is not defined

In [ ]:
my_shape_frequency = pd.read_parquet("./data/test/shape_frequency.parquet")

In [ ]:
my_shape_frequency[(my_shape_frequency.calitp_itp_id.isin(debug.calitp_itp_id)) &
                   (my_shape_frequency.route_id.isin(debug.route_id) & 
                    (my_shape_frequency.mean_runtime_min.isna())
                   )]

In [ ]:

my_shape_frequency.trips_per_hour.describe()

#### Eric Tries Stuff